In [6]:
import pandas as pd
from pathlib import Path
from Bio import SeqIO
from dna_features_viewer import GraphicFeature, GraphicRecord

Functions for reading sequence data

In [33]:
def recursive_seq_read(path):
    for thing in Path(path).iterdir():
        if thing.is_dir():
            recursive_seq_read(thing)
        else:
            try:
                yield seq_read(thing)
            except ValueError:
                # print(f'Could not read {thing}')
                pass
            
def seq_read(filepath):
    common_suffi = {
        '.fa': 'fasta'
    }
    fp = Path(filepath)
    if fp.suffix in common_suffi:
        record_format = common_suffi[fp.suffix]
    else:
        record_format = fp.suffix[1:]  # use file suffix without . for type
    
    return SeqIO.parse(str(fp), record_format)  

Read Albert's predictions.

In [34]:
NEG_PRED_PATH = 'albertPredict/negative_predictions_ethan.csv'
POS_PRED_PATH = 'albertPredict/positive_predictions_ethan.csv'

In [24]:
pos_strand_pred, neg_strand_pred = pd.read_csv(POS_PRED_PATH), pd.read_csv(NEG_PRED_PATH)

## T7 initiation series predictions

Pull out T7 initiation series rows.

In [27]:
def get_t7_init_pred(df):
    return df[df['name'].str.contains('T7_init')]

t7_init_pos = get_t7_init_pred(pos_strand_pred)
t7_init_pos

,name,cluster,0,1,2,3,4,5,6,7,...,3029,3030,3031,3032,3033,3034,3035,3036,3037,3038


Hmm, maybe there are missing sequences?

In [35]:
print(pos_strand_pred.shape)
print(neg_strand_pred.shape)

(961, 3041)
(961, 3041)


Check number of sequences in the substrate file I sent.

In [37]:
len(list(seq_read('prediction_substrates_2021-11-14.fa')))

994

In [39]:
994-961

33

Does look like some are missing, maybe not included becuase these are different in length?